In [1]:
from matplotlib.colors import ListedColormap # for grgphing decision boundaries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
data_folder = 'data'
X_train = pd.read_csv(f'./{data_folder}/X_train.csv')
y_train = pd.read_csv(f'./{data_folder}/y_train.csv')
X_test = pd.read_csv(f'./{data_folder}/X_test.csv')
y_test = pd.read_csv(f'./{data_folder}/y_test.csv')

In [3]:
class Network:
    def __init__(self):
        # layers/activations
#         self.input = np.random.rand(4, 1)
        self.h1 = np.random.rand(2, 1)
        self.h2 = np.random.rand(3, 1)
        
        # weights 
        self.w_1 = np.random.rand(2, 4)
        self.w_2 = np.random.rand(3, 2)
        
        # biases 
        self.b_1 = np.random.rand(2, 1)
        self.b_2 = np.random.rand(3, 1)


    def relu(self,activations):
        return np.maximum(0, activations)

    def relu_deriv(self, activations):
        return activations > 0

    def softmax(self, activations):
        return np.exp(activations) / np.sum(np.exp(activations))
    
    def feed_forward(self, X):
        # input
        # reshapre the input into vector form.
        # Example [1, 0, 0] -> [[1], [1], [1]]
        input_ = np.reshape(X, (-1,1))
        
        # input -> h1
        # the activations in the first hidden layer are given by the dot product 
        # of the weights by the input plus some biass its all then passed into
        # our activation function. relu(W_1*x+b_1)
        h1_activations = self.relu(np.dot(self.w_1, input_) + self.b_1)
        self.h1 = h1_activations

        # h1 -> h2
        # the activations in the seocnd hidden layer (h_2) are given by the dot product 
        # of the second weights (w_2) by the previous activations (h1) plus the bias(b_2).
        # W_2*h1+b_2
        h2_activations = (np.dot(self.w_2, h1_activations) + self.b_2)
        self.h2 = h2_activations 

        # h2 -> output 
        # our output activtions/predictions are given by the second layer activations (h_2)
        # put into the softmax function. 
        output = self.softmax(self.h2)
        
        return (output, max(output))
    
    def back_prop(self, output, y, input_, learning_rate=0.01):
        y = np.reshape(y, (-1,1))
        
        
        # Derivative of the cost function.
        # The cost function is C = (output - y)^2
        # The derivative of that is 2 (output - y)
        dc = 2 * (output - y)
        
        # Derivative of the cost function with respect to the weights (w_2).
        # The because our output is given by:output = w_2*h_1+b_2. The deriative
        # of the cost function with respect to w_2 is: dc * h1
        dw2 = dc.dot(self.h1.T)
        
        dc = (2 * (output - y))
        dc_dw2 = -learning_rate * dc.dot(self.h1.T)
        b_2 = -learning_rate * dc
        
        h1_error = self.w_2.T.dot(dc) * self.relu_deriv(self.h1)
        dc_dw1 = -learning_rate * (h1_error.dot(input_.T))
        b_1 = -learning_rate * h1_error
        
        self.w_1 +=  dc_dw1
        self.w_2 +=  dc_dw2
        
        self.b_1 += b_1
        self.b_2 += b_2
        return 0
       

    def train(self, X, Y, itterations):        
        y_dummies = pd.get_dummies(Y)
        data = pd.concat([X, y_dummies], axis=1)
        for i in range(itterations):
            for index, row in data.iterrows():
                # select the label
                y = row.tolist()[4:]
                # select the x
                x = row.tolist()[:4] 
                output_activations, prediction = self.feed_forward(x)
                self.back_prop(output_activations, y, x)    

In [4]:
new_class = Network()

In [5]:
new_class.train(X_train, y_train, 10)

input: [[5.5]
 [2.4]
 [3.7]
 [1. ]]

h1: [[11.30919617]
 [10.1443441 ]]

h2: [[13.29281276]
 [13.92092132]
 [10.01014081]]

output: [[0.3434549 ]
 [0.64365597]
 [0.01288912]]
input: [[4.8]
 [3. ]
 [1.4]
 [0.1]]

h1: [[8.09605791]
 [7.27456071]]

h2: [[ 8.52952236]
 [11.16641001]
 [ 7.34037367]]

output: [[0.06547014]
 [0.91459548]
 [0.01993437]]
input: [[5.5]
 [2.6]
 [4.4]
 [1.2]]

h1: [[12.00543937]
 [10.91782391]]

h2: [[15.81913702]
 [13.3605997 ]
 [10.52546868]]

output: [[0.9169405 ]
 [0.07845343]
 [0.00460607]]
input: [[5. ]
 [3.2]
 [1.2]
 [0.2]]

h1: [[8.15617391]
 [7.27503403]]

h2: [[ 7.52103426]
 [12.31302034]
 [ 7.29675368]]

output: [[0.00817397]
 [0.98529429]
 [0.00653174]]
input: [[6.9]
 [3.1]
 [5.1]
 [2.3]]

h1: [[14.9499264 ]
 [13.38480428]]

h2: [[17.76648651]
 [18.22497412]
 [12.75543921]]

output: [[0.38634741]
 [0.611078  ]
 [0.00257459]]
input: [[5.9]
 [3. ]
 [5.1]
 [1.8]]

h1: [[12.99119638]
 [11.86423003]]

h2: [[12.96275986]
 [11.62132629]
 [18.36571596]]

outpu